In [28]:
import os
import numpy as np
import pandas as pd
import spacy
import re

In [2]:
pwd

'/Users/hao/workspace/hpi-de/2nd_Semester/textVis/TextVisualisierungProject'

In [62]:
bund_csv = './Data/Results/Bundestagswahlen.csv'
euro_csv = './Data/Results/Europawahlen.csv'
bund_manifestos_dir = './Data/Manifestos/Bundestagswahlen/'
euro_manifestos_dir = './Data/Manifestos/Europawahlen/'

In [59]:
bund_results_df = pd.read_csv(bund_csv)
bund_results_df.set_index('Partei', inplace=True)
bund_results_df

,1949,1953,1957,1961,1965,1969,1972,1976,1980,1983,1987,1990,1994,1998,2002,2005,2009,2013,2017
Partei,,,,,,,,,,,,,,,,,,,
CDUCSU,"31,01","45,17","50,19","45,32","47,59","46,09","44,86","48,63","44,54","48,79","44,26","43,82","41,43","35,14","38,51","35,17","33,8","41,55","32,93"
SPD,"29,22","28,84","31,75","36,22","39,28","42,67","45,85","42,56","42,86","38,18","37,04","33,46","36,39","40,93","38,52","34,25","23,03","25,74","20,51"
FDP,"11,92","9,54","7,71","12,77","9,49","5,77","8,36","7,92","10,62","6,95","9,09","11,03","6,92","6,25","7,37","9,83","14,56","4,76","10,75"
Gruene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,5","5,57","8,26","5,05","7,27","6,7","8,56","8,12","10,71","8,44","8,94"
Linke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,43","4,39","5,1","3,99","8,71","11,89","8,59","9,24"
AFD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,7","12,64"
Wahlbeteiligung,"78,49","85,99","87,77","87,74","86,77","86,67","91,11","90,75","88,57","89,09","84,33","77,76","78,97","82,2","79,08","77,65","70,78","71,55","76,16"


In [64]:
euro_results_df = pd.read_csv(euro_csv)
euro_results_df.set_index('Partei', inplace=True)
euro_results_df

,1979,1984,1989,1994,1999,2004,2009,2014,2019
Partei,,,,,,,,,
CDUCSU,"49,2",46,"37,7","38,8","48,7","44,5","37,9","35,3","28,9"
SPD,"40,8","37,4","37,3","32,2","30,7","21,5","20,8","27,3","15,8"
FDP,6,"4,8","5,6","4,1",3,"6,1",11,"3,4","5,4"
Gruene,"3,2","8,2","8,4","10,1","6,4","11,9","12,1","10,7","20,5"
Linke,NaN,NaN,NaN,"4,7","5,8","6,1","7,5","7,4","5,5"
AFD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7,1",11
Wahlbeteiligung,"65,7","56,8","62,3",60,"45,2",43,"43,3","48,1","61,4"


In [78]:
texts = []
election_types = []
election_results = []
party_names = []
participations = []
election_years = []

for (dirpath, dirnames, filenames) in os.walk('./Data/Manifestos/'):
    for filename in filenames:
        election_type = filename[0]
        election_year = filename[1:5]
        party_name = filename.split('_')[1].split('.')[0]
        
        if election_type == 'b':
            text_df = pd.read_csv(os.path.join(dirpath, filename))
            text_df = text_df[~text_df.iloc[:, 0].isnull()]
            
            if len(text_df) > 1:
                text = ' '.join(text_df.iloc[:, 0].values)
            else:
                text = text_df.iloc[:, 0].values[0]
            
            election_result = float(str(bund_results_df.loc[party_name][election_year]).replace(',', '.'))
            participation = float(str(bund_results_df.loc['Wahlbeteiligung'][election_year]).replace(',', '.'))
        
        if election_type == 'e':
            pdf = parser.from_file(filename=os.path.join(dirpath, filename))
            pdf_sites = int(pdf['metadata']['xmpTPg:NPages'])

            if pdf['content'] is not None:
                text = clean_text(pdf['content'])

                if len(text) <= pdf_sites * 200: # Threshhold 
                    text = clean_text(get_ocr_text(os.path.join(dirpath, filename)))
            else:
                text = clean_text(get_ocr_text(os.path.join(dirpath, filename)))
            election_result = float(str(euro_results_df.loc[party_name][election_year]).replace(',', '.'))
            participation = float(str(euro_results_df.loc['Wahlbeteiligung'][election_year]).replace(',', '.'))
            
            
        texts.append(text)
        election_types.append(election_type)
        election_results.append(election_result)
        party_names.append(party_name)
        participations.append(participation)
        election_years.append(election_year)

In [79]:
df = pd.DataFrame({
    'year': election_years,
    'participation': participations,
    'party_name': party_names,
    'result': election_results,
    'text': texts,
    'type': election_types
})

In [86]:
df = df.sort_values(by=['type', 'year', 'party_name']).reset_index(drop=True)

In [88]:
df[df['type'] == 'e']

,year,participation,party_name,result,text,type
79,1979,65.7,CDUCSU,49.2,Politisches Programm Fr der x * Europäischen D...,e
80,1979,65.7,FDP,6.0,Brief Halle Wahlprogramm zur Europawahl 1979 ...,e
81,1979,65.7,Gruene,3.2,Die Grünen - Alternative für Europa DE o TISC...,e
82,1979,65.7,SPD,40.8,Dokumente Soziale Demokratie für Europa Progra...,e
83,1984,56.8,CDUCSU,46.0,AKTIONSPROGRAMM DER EUROPÄISCHEN VOLKSPARTEI ...,e
84,1984,56.8,FDP,4.8,Brief Halle Wahlprogramm zur Europawahl 1984 ...,e
85,1984,56.8,Gruene,8.2,Die Grünen - Global Denken - vor Ort handeln!...,e
86,1984,56.8,SPD,37.4,"os 2082 os Bo) e aa"" 83 2802 0929 Set DIN 19.0...",e
87,1989,62.3,CDUCSU,37.7,_Europa Im Bliek#eld ~~~~f:!~ Aus der Arbert ...,e
88,1989,62.3,FDP,5.6,"F.i••"",. Ho...o•• STIFTUNG FÜR DIE FREIHEIT L...",e


In [89]:
df.to_csv('./Data/aggregated_manifestos.csv')

In [34]:
def get_spacy_doc(text):
    nlp = spacy.load('de')
    doc=nlp(text)
    return doc    

In [33]:
nlp = spacy.load('de')

In [38]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 

In [54]:
# Import libraries 
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 

# Path of the pdf 
PDF_file = "./Data/Manifestos/Europawahlen/e1979_SPD.pdf"

''' 
Part #1 : Converting PDF to images 
'''

# Store all the pages of the PDF in a variable 
pages = convert_from_path(PDF_file, 500) 

# Counter to store images of each page of PDF to image 
image_counter = 1

# Iterate through all the pages stored above 
for page in pages: 

	# Declaring filename for each page of PDF as JPG 
	# For each page, filename will be: 
	# PDF page 1 -> page_1.jpg 
	# PDF page 2 -> page_2.jpg 
	# PDF page 3 -> page_3.jpg 
	# .... 
	# PDF page n -> page_n.jpg 
	filename = "page_"+str(image_counter)+".jpg"
	
	# Save the image of the page in system 
	page.save(filename, 'JPEG') 

	# Increment the counter to update filename 
	image_counter = image_counter + 1

''' 
Part #2 - Recognizing text from the images using OCR 
'''
# Variable to get count of total number of pages 
filelimit = image_counter-1

# Creating a text file to write the output 
outfile = "out_text.txt"

# Open the file in append mode so that 
# All contents of all images are added to the same file 
f = open(outfile, "a") 

# Iterate from 1 to total number of pages 
for i in range(1, filelimit + 1): 

	# Set filename to recognize text from 
	# Again, these files will be: 
	# page_1.jpg 
	# page_2.jpg 
	# .... 
	# page_n.jpg 
	filename = "page_"+str(i)+".jpg"
		
	# Recognize the text as string in image using pytesserct 
	text = str(((pytesseract.image_to_string(Image.open(filename), lang='deu', config='--psm 6')))) 

	# The recognized text is stored in variable text 
	# Any string processing may be applied on text 
	# Here, basic formatting has been done: 
	# In many PDFs, at line ending, if a word can't 
	# be written fully, a 'hyphen' is added. 
	# The rest of the word is written in the next line 
	# Eg: This is a sample text this word here GeeksF- 
	# orGeeks is half on first line, remaining on next. 
	# To remove this, we replace every '-\n' to ''. 
	text = text.replace('-\n', '')	 

	# Finally, write the processed text to the file. 
	f.write(text) 

# Close the file after writing all the text. 
f.close() 

In [72]:
def clean_text(text, lower=False, remove_numbers=False, remove_punctations=False):
    prep_text = text.replace('\n',' ')
    prep_text = prep_text.replace('\t',' ')
    prep_text = re.sub(' +', ' ', prep_text)
    
    if lower:
        prep_text = prep_text.lower()
    
    if remove_numbers:
        prep_text = re.sub(r'\d+', '', prep_text)
    
    if remove_punctations:
        prep_text = prep_text.translate(string.punctuation)
        
    return prep_text

In [73]:
def get_ocr_text(PDF_file):
    image_counter = 1
    pages = convert_from_path(PDF_file, 500)
    
    for page in pages:
        filename = "page_"+str(image_counter)+".jpg"
        page.save(filename, 'JPEG') # Save the image of the page in system 
        image_counter = image_counter + 1
    
    text_concat = ''
    filelimit = image_counter-1
    
    for i in range(1, filelimit + 1):
        filename = "page_"+str(i)+".jpg"
        # Recognize the text as string in image using pytesserct 
        text = str(((pytesseract.image_to_string(Image.open(filename), lang='deu'))))
        text = text.replace('-\n', '')
        text_concat = text_concat + text
    
    return text_concat

In [6]:
from tika import parser

In [41]:
for (dirpath, dirnames, filenames) in os.walk(euro_manifestos_dir):
    for filename in filenames:
        pdf = parser.from_file(filename=os.path.join(dirpath, filename))
        pdf_sites = int(pdf['metadata']['xmpTPg:NPages'])
        
        if pdf['content'] is not None:
            text = clean_text(pdf['content'])
            
            if len(text) <= pdf_sites * 200:
                print(filename)
                #print(text_obj['content'])
        else:
            print(filename)
            #print(text_obj['content'])

e1989_SPD.pdf
e1994_Linke.pdf
e1979_SPD.pdf
e1984_SPD.pdf
e1979_CDUCSU.pdf


In [34]:
parser.from_file(filename='./Data/Manifestos/Europawahlen/e1989_SPD.pdf')

{'status': 200,
 'content': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\tIndex\n\tEinleitung\n\tWir wollen ein geeintes Europa\n\tWir wollen ein wirtschaftlich starkes Europa\n\tWir wollen ein solidarisches Europa\n\tWir wollen ein bewohnbares Europa\n\tWir wollen ein demokratisches Europa\n\tEuropa hat eine aktive Rolle in der Welt zu spielen\n\tVorbehalte gegen das Programm für 1989\n\n',
 'metadata': {'Content-Type': 'application/pdf',
  'Creation-Date': '2004-03-24T13:23:11Z',
  'Last-Modified': '2004-03-24T13:26:20Z',
  'Last-Save-Date': '2004-03-24T13:26:20Z',
  'X-Parsed-By': ['org.apache.tika.parser.DefaultParser',
   'org.apache.tika.parser.pdf.PDFParser'],
  'X-TIKA:parse_time_millis': '8',
  'access_permission:assemble_document': 'true',
  'access_permission:can_modify': 'true',
  'access_permission:can_print': 'true',
  'access_permission

In [16]:
parser.from_file(filename='./Data/Manifestos/Europawahlen/e2019_Gruene.pdf')

{'status': 200,
 'content': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEuropawahlprogramm 2019\n\nEuropas \nVer  spre chen \ner neuern.\n\n\n\nDieses Europawahlprogramm wurde auf \nder 43. Bundesdelegiertenkonferenz von \nBÜNDNIS 90/DIE GRÜNEN vom 9. bis 11. November 2018 \nin Leipzig beschlossen.\n\nHerausgeber*in:\nBÜNDNIS 90/DIE GRÜNEN \nPlatz vor dem Neuen Tor 1 \n10115 Berlin\nTelefon: 030 28442-0 \nFax: 030 28442-210 \nE-Mail: info@gruene.de \nInternet: www.gruene.de \n\nV.\u2009i.\u2009S.\u2009d.\u2009P.:  \nBÜNDNIS 90/DIE GRÜNEN  \nNicolas Schwendemann \nPlatz vor dem Neuen Tor 1 \n10115 Berlin \n\nTitelgestaltung, Layout und Satz: \nKOMPAKTMEDIEN Agentur für  \nKommunikation GmbH, Berlin  \nwww.kompaktmedien.de \nDruck: Druckhaus Berlin-Mitte\n\n\n\nEUROPAWAHLPROGRAMM 2019\n\n\n\nEUROPAS VERSPRECHEN ERNEUERN.\n\nEuropawahlprogramm 2019\n\nBÜNDNIS 90/DIE GRÜNEN\n\n2\n\nIn\nha\n\nlt\n\nPRÄAMBEL 7\n\n1 ERHALTEN, WAS UNS ERHÄLT:  \nUNS

In [19]:
len('Europas \nVer  spre chen \ner neuern.\n\n\n\nDieses Europawahlprogramm wurde auf \nder 43. Bundesdelegiertenkonferenz von \nBÜNDNIS 90/DIE GRÜNEN vom 9. bis 11.')

152

In [20]:
len('\n\n\n\n\n\n\n\n\n\n\n\n\n\n')

14

In [58]:
get_ocr_text('./Data/Manifestos/Europawahlen/e1979_SPD.pdf')

Dokumente

 

Soziale Demokratie
für Europa
Programm der

Sozialdemokratischen Partei Deutschlands
für die erste europäische Direktwahl 1979

SPD
_ Der Europa-Parteitag der SPD in Köln hat am 10. Dezember
1978 das nachfolgende Programm einstimmig verabschiedet:

Inhaltsverzeichnis
Seite
1. Der demokratische Sozialismus als Idee
europäischer Humanität 7
ne
1.1 Ein neues Kapitel europäischer Geschichte 7
1.2 Frieden, Humanität, Demokratie, Solidarität 8
1.3 Freiheitsrechte für alle Menschen 10
1.4 Die Sozialdemokraten in Europa
kämpfen gemeinsam 12

14.5 Wir streiten für die Humanisierung der Arbeit
durch Demokratie in der Wirtschaft 15

 

-H. Soziale Demokratie als Fundament der

europäischen Gesellschaft 17
u NE
:1.1 Für eine Gemeinschaft der Vollbeschäftigung 17
ee ie ee TEE LE

.2  Arbeitsproduktivität, Produktionswachstum

| und Arbeitszeit 23
1.3 Die 35-Stunden-Woche als Ziel unserer Politik 24
ee ER EIN ERBETEN U TEE

3
1.4
11.5
11.6
1.7
11.8

m.

111.1

1.2

118.3

111.4
111.5
I

die SPD 1925 in ihrem Heidelberger Programm die Vereinigten Staaten von Europa fordern. Auch während des
Widerstandskampfes gegen Faschismus und Nationalsozialismus haben sich die Sozialdemokraten von diesem

Ziel nicht abbringen lassen.

Die politischen, die nationalistischen, die weltanschaulichen
Zerklüftungen überwindend, hat die SPD 1959 in ihrem
Godesberger Programm die Verwurzelung des demokratischen Sozialismus in Europa in christlicher Ethik, im
Humanismus und in der klassischen Philosophie aufgezeigt.

Die Entscheidung für den Sozialismus kann unterschiedlich
begründet werden. Die Übereinstimmung demokratischer
Sozialisten wurzeit nicht in einer einheitlichen religiösen,
philosophischen oder wissenschaftlichen Anschauung,
sondern in gleichen politischen Zielen, die auf gemeinsamen sittlichen Grundwerten beruhen. Diese Grundwerte
sind Freiheit, Gerechtigkeit und Solidarität. Die politischgesellschaftlichen Grundforderungen des demokratischen
Sozialismus ergeben sich aus der En

einzelstaatliche Versuch zur Bekämpfung der Arbeitslosigkeit, der den Politiken der übrigen Länder nicht Rechnung
trägt und sich auf eine Konjunkturpolitik als alleiniges
Instrument des Krisenmanagements beschränkt, hat
keine Aussicht auf Erfolg. Da die Gemeinschaft selbst

in nur sehr beschränktem Maße über eigene Instrumente
der allgemeinen Wirtschaftspolitik verfügt, ist eine wesentlich stärkere Koordinierung der nationalen Politiken zur
Bekämpfung der Arbeitslosigkeit nötig.

Nicht nur 6 Mio. Arbeitsiose, sondern geburtenstarke
Jahrgänge erwarten in den nächsten 10 Jahren in der
Gemeinschaft einen Arbeitsplatz. Auch der Anteil der
Frauen an der Nachfrage nach Arbeit wächst in den
Ländern der Gemeinschaft. Allein auf die „Selbstheilungskräfte des Marktes“ zu vertrauen und außer acht zu
lassen, daß wir mit einem gegenüber früher verringerten
Wachstum, aber gleichzeitig mit höheren Zuwachsraten
der Produktivität und einem verschärften Tempo der
Einführung neuer Technologien und der Ra

KeyboardInterrupt: 

e1989_SPD.pdf
e1994_Linke.pdf
e1979_SPD.pdf
e1984_SPD.pdf
e1979_CDUCSU.pdf